In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 23.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 24.8 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 2.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 48.9 MB/s eta 0:00:0000:0100:01
  Attempting uninsta

In [2]:

# Standard libraries
import os
import shutil
import random
import math
import json
from pathlib import Path
from collections import Counter, defaultdict
from typing import List, Tuple, Dict, Optional

# Third-party libraries
import cv2
import torch
import torchvision.transforms as T
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
from tabulate import tabulate  # For nice tables


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

True
Tesla P100-PCIE-16GB


In [4]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

# --- Augmentation setup ---
train_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.5),
    A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=30, val_shift_limit=20, p=0.5),
    A.GaussianBlur(blur_limit=(3,7), p=0.3),
    A.Resize(640, 640)
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

# --- Paths ---
BASE_PATH = Path("/kaggle/input/weed-detection-dataset/labeled")
OUTPUT_PATH = Path("/kaggle/working/augmented_dataset")
OUTPUT_IMG = OUTPUT_PATH / "images"
OUTPUT_ANN = OUTPUT_PATH / "annotations"
OUTPUT_IMG.mkdir(parents=True, exist_ok=True)
OUTPUT_ANN.mkdir(parents=True, exist_ok=True)

# --- Augmentation ---
for img_path in (BASE_PATH / "images").glob("*.jpg"):
    ann_path = BASE_PATH / "annotations" / f"{img_path.stem}.txt"
    if not ann_path.exists():
        continue

    # Read image and annotations
    image = cv2.imread(str(img_path))
    with open(ann_path, 'r') as f:
        bboxes = [list(map(float, line.strip().split())) for line in f]
        class_labels = [int(b[0]) for b in bboxes]
        bbox_coords = [b[1:] for b in bboxes]

    # Apply multiple augmentations
    for i in range(3):  # Number of augmented versions per image
        augmented = train_transform(image=image, bboxes=bbox_coords, class_labels=class_labels)
        aug_image = augmented['image']
        aug_bboxes = augmented['bboxes']
        aug_labels = augmented['class_labels']

        # Save image
        aug_img_name = f"{img_path.stem}_aug{i}.jpg"
        cv2.imwrite(str(OUTPUT_IMG / aug_img_name), aug_image)

        # Save annotations
        aug_ann_name = f"{img_path.stem}_aug{i}.txt"
        with open(OUTPUT_ANN / aug_ann_name, 'w') as f:
            for lbl, bbox in zip(aug_labels, aug_bboxes):
                f.write(f"{lbl} " + " ".join(map(str, bbox)) + "\n")

# --- Restructure dataset ---
FINAL_PATH = Path("/kaggle/working/final/Dataset")
TRAIN_PATH = FINAL_PATH / "Train"
DEV_PATH = FINAL_PATH / "Dev"
UNLABELED_PATH = FINAL_PATH / "Unlabelled"

# Create directories
for folder in [TRAIN_PATH / "images", TRAIN_PATH / "annotations",
               DEV_PATH / "images", DEV_PATH / "annotations",
               UNLABELED_PATH]:
    folder.mkdir(parents=True, exist_ok=True)

# Move augmented images to Train
for subdir in ["images", "annotations"]:
    src_path = OUTPUT_PATH / subdir
    dest_path = TRAIN_PATH / subdir
    for file in src_path.glob("*"):
        shutil.move(str(file), str(dest_path / file.name))

# Copy test and unlabeled data
shutil.copytree(BASE_PATH.parent / "test", DEV_PATH, dirs_exist_ok=True)
shutil.copytree(BASE_PATH.parent / "unlabeled", UNLABELED_PATH, dirs_exist_ok=True)

# --- Prepare YOLO dataset ---
YOLO_DIR = Path("/kaggle/working/yolo_dataset")
IMG_TRAIN_DIR = YOLO_DIR / "images" / "train"
IMG_VAL_DIR = YOLO_DIR / "images" / "val"
LBL_TRAIN_DIR = YOLO_DIR / "labels" / "train"
LBL_VAL_DIR = YOLO_DIR / "labels" / "val"

for folder in [IMG_TRAIN_DIR, IMG_VAL_DIR, LBL_TRAIN_DIR, LBL_VAL_DIR]:
    folder.mkdir(parents=True, exist_ok=True)

# Combine Train + Dev images for split
train_images = list((TRAIN_PATH / "images").glob("*.jpg")) + list((DEV_PATH / "images").glob("*.jpg"))
random.seed(42)
random.shuffle(train_images)

split_idx = int(0.8 * len(train_images))
train_files = train_images[:split_idx]
val_files = train_images[split_idx:]

def copy_files(image_list, img_dest, lbl_dest, src_img_dir, src_lbl_dir):
    for img_path in image_list:
        lbl_path = src_lbl_dir / f"{img_path.stem}.txt"
        if lbl_path.exists():
            shutil.copy(img_path, img_dest)
            shutil.copy(lbl_path, lbl_dest)

copy_files(train_files, IMG_TRAIN_DIR, LBL_TRAIN_DIR, TRAIN_PATH / "images", TRAIN_PATH / "annotations")
copy_files(val_files, IMG_VAL_DIR, LBL_VAL_DIR, TRAIN_PATH / "images", TRAIN_PATH / "annotations")
copy_files(train_files, IMG_TRAIN_DIR, LBL_TRAIN_DIR, DEV_PATH / "images", DEV_PATH / "annotations")
copy_files(val_files, IMG_VAL_DIR, LBL_VAL_DIR, DEV_PATH / "images", DEV_PATH / "annotations")

# --- Create YAML ---
yaml_content = f"""
train: {IMG_TRAIN_DIR.resolve()}
val: {IMG_VAL_DIR.resolve()}
nc: 2
names: ['Crop', 'Weed']
"""
(YOLO_DIR / "data.yaml").write_text(yaml_content)

print("✅ Dataset preparation complete! YOLO-ready dataset is ready for training.")


✅ Dataset preparation complete! YOLO-ready dataset is ready for training.


In [41]:
from ultralytics import YOLO

# Load pretrained YOLO11m model
model = YOLO("yolo12m.pt")

# Training configuration
train_params = {
    "data": "/kaggle/working/yolo_dataset/data.yaml",  # must include 'train' and 'val'
    "epochs": 60,          # early stopping
    "imgsz": 640,
    "batch": 16,
    "device": "cuda",
    "optimizer": "SGD",
    "lr0": 0.01,
    "lrf": 0.1,
    "weight_decay": 0.0005,
    "momentum": 0.937,
    "workers": 8,
    "project": "runs/train",
    "name": "yolo11m_augmented_dataset",
    "verbose": True,
    "save": True,
}

# Start training
model.train(**train_params)


Ultralytics 8.3.203 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/yolo_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=60, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.1, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo12m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11m_augmented_dataset6, nbs=64, nms=False, opset=None, optimize=False, optimizer=SGD, overlap_mask=True, patience=10

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        130        214      0.906      0.897      0.955      0.687
                  Crop         60        115      0.874      0.896      0.942      0.706
                  Weed         70         99      0.939      0.899      0.967      0.668
Speed: 0.2ms preprocess, 15.6ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to /kaggle/working/runs/train/yolo11m_augmented_dataset6


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f84a909a4d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [42]:
model.save('v11m_50_epochs.pt')

In [8]:

# List your trained models
models_info = [
    {"name": "Base", "path": "/kaggle/working/v11m_50_epochs.pt"},
    # Add more models here if needed
]

# Path to your dataset YAML
data_yaml = "/kaggle/working/yolo_dataset/data.yaml"

# List to store metrics for comparison
comparison_metrics = []

for info in models_info:
    print(f"\nEvaluating {info['name']}...")
    model = YOLO(info["path"]).to("cuda")
    model.eval()
    
    # Run validation
    results = model.val(data=data_yaml)
    res_dict = results.results_dict
    
    # Extract metrics safely
    precision = res_dict.get('metrics/precision(B)', 0)
    recall = res_dict.get('metrics/recall(B)', 0)
    mAP_50_95 = res_dict.get('metrics/mAP50-95(B)', 0)
    
    # Compute F1
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    combined_metric = 0.5 * f1_score + 0.5 * mAP_50_95
    
    comparison_metrics.append({
        "Model": info["name"],
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1_score,
        "mAP50-95": mAP_50_95,
        "Combined": combined_metric
    })

# Print comparative table
print("\n=== Comparative Analysis ===")
print(tabulate(comparison_metrics, headers="keys", tablefmt="fancy_grid", floatfmt=".4f"))



Evaluating Base...
YOLOv12m summary (fused): 169 layers, 20,106,454 parameters, 0 gradients, 67.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2521.0±611.4 MB/s, size: 157.2 KB)
val: Scanning /kaggle/working/yolo_dataset/labels/val.cache... 130 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 130/130 188.6Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 9/9 2.3it/s 4.0s0.4ss


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        130        214       0.89      0.922      0.954      0.734
                  Crop         60        115       0.85      0.913      0.947      0.756
                  Weed         70         99      0.929      0.931      0.961      0.712
Speed: 2.9ms preprocess, 17.9ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to /kaggle/working/runs/detect/val

=== Comparative Analysis ===
╒═════════╤═════════════╤══════════╤════════════╤════════════╤════════════╕
│ Model   │   Precision │   Recall │   F1-Score │   mAP50-95 │   Combined │
╞═════════╪═════════════╪══════════╪════════════╪════════════╪════════════╡
│ Base    │      0.8896 │   0.9220 │     0.9055 │     0.7343 │     0.8199 │
╘═════════╧═════════════╧══════════╧════════════╧════════════╧════════════╛


In [10]:
import shutil

# after training
best_model_path = "runs/train/yolo11m_augmented_dataset/weights/best.pt"
shutil.copy(best_model_path, "v11m_50_epochs.pt")


'v11m_50_epochs.pt'

In [11]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = YOLO("v11m_50_epochs.pt")
model.model.to(device)

weak_transform = T.Compose([
    T.RandomHorizontalFlip(p=0.5),
    T.RandomResizedCrop(size=640, scale=(0.8, 1.0))
])

strong_transform = T.Compose([
    T.RandomHorizontalFlip(p=0.5),
    T.RandomResizedCrop(size=640, scale=(0.5, 1.0)),
    T.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
    T.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 2.0))
])

UNLABELED_DIR = "/kaggle/working/final/Dataset/Unlabelled"
image_files = [f for f in os.listdir(UNLABELED_DIR) if f.endswith(".jpg")]

optimizer = torch.optim.Adam(model.model.parameters(), lr=5e-5)
for img_file in image_files:
    img_path = os.path.join(UNLABELED_DIR, img_file)
    img = cv2.imread(img_path)
    img_pil = T.ToPILImage()(img)

    weak_img = weak_transform(img_pil)
    strong_img = strong_transform(img_pil)
    weak_tensor = T.ToTensor()(weak_img).unsqueeze(0).to(device).requires_grad_(True)
    strong_tensor = T.ToTensor()(strong_img).unsqueeze(0).to(device).requires_grad_(True)

    model.model.to(device)
    weak_preds = model.model(weak_tensor)  # Get raw output logits
    strong_preds = model.model(strong_tensor)

    if isinstance(weak_preds, tuple):
        weak_preds = weak_preds[0]
        strong_preds = strong_preds[0]

    #Consistency loss
    loss = torch.nn.functional.mse_loss(weak_preds, strong_preds)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

model.save("model_consistency.pt")

In [12]:

UNLABELED_DIR = "/kaggle/working/final/Dataset/Unlabelled"
TRAIN_LABELS_DIR = "/kaggle/working/yolo_dataset/labels/train"
DATA_YAML = "/kaggle/working/yolo_dataset/data.yaml"

# Directory for pseudo labels
PSEUDO_LABELS_DIR = "/kaggle/working/yolo_dataset/labels/pseudo"
os.makedirs(PSEUDO_LABELS_DIR, exist_ok=True)

# -----------------------------
# Step 1: Load fine-tuned model
# -----------------------------
model = YOLO("model_consistency.pt")  # model from consistency training

# -----------------------------
# Step 2: Predict pseudo labels
# -----------------------------
CONF_THRESHOLD = 0.85  # confidence threshold

results = model.predict(source=UNLABELED_DIR, save=False)

for result in results:
    img_name = os.path.basename(result.path).replace(".jpg", ".txt")
    label_path = os.path.join(PSEUDO_LABELS_DIR, img_name)

    with open(label_path, "w") as f:
        for box in result.boxes.data:
            cls, x_center, y_center, width, height, conf = box.tolist()
            if conf > CONF_THRESHOLD:
                f.write(f"{int(cls)} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

# -----------------------------
# Step 3: Merge pseudo labels with train labels
# -----------------------------
for file in os.listdir(PSEUDO_LABELS_DIR):
    if file.endswith(".txt"):
        shutil.copy(os.path.join(PSEUDO_LABELS_DIR, file), TRAIN_LABELS_DIR)

# -----------------------------
# Step 4: Retrain YOLO on combined dataset
# -----------------------------
train_params = {
    "data": "/kaggle/working/yolo_dataset/data.yaml",  # must include 'train' and 'val'
    "epochs": 150,
    "patience": 10,          # early stopping
    "imgsz": 640,
    "batch": 16,
    "device": "cuda",
    "optimizer": "SGD",
    "lr0": 0.01,
    "lrf": 0.1,
    "weight_decay": 0.0005,
    "momentum": 0.937,
    "workers": 8,
    "project": "runs/train",
    "name": "yolo11m_augmented_dataset",
    "verbose": True,
    "save": True,
}

model.train(**train_params)

# -----------------------------
# Step 5: Load & save best model
# -----------------------------
model.save("model_pseudo.pt")

print(f"Training finished. Best model saved at: model_pseudo.pt")



image 1/1000 /kaggle/working/final/Dataset/Unlabelled/agri_0_1009_jpeg.rf.bd6418200779c7138320a9dd87b37a01.jpg: 640x640 1 Weed, 20.8ms
image 2/1000 /kaggle/working/final/Dataset/Unlabelled/agri_0_1017_jpeg.rf.10421009c95805e81d82a2b6a8c3b116.jpg: 640x640 1 Weed, 20.9ms
image 3/1000 /kaggle/working/final/Dataset/Unlabelled/agri_0_1018_jpeg.rf.30238c6d22d384618e1ef4297dea6e34.jpg: 640x640 1 Weed, 20.7ms
image 4/1000 /kaggle/working/final/Dataset/Unlabelled/agri_0_1020_jpeg.rf.26add55029735d0e6ae4ec0a57bb210a.jpg: 640x640 1 Weed, 19.3ms
image 5/1000 /kaggle/working/final/Dataset/Unlabelled/agri_0_1024_jpeg.rf.f163741f6c70d41a5a88d6f1297d19a8.jpg: 640x640 1 Weed, 19.2ms
image 6/1000 /kaggle/working/final/Dataset/Unlabelled/agri_0_1026_jpeg.rf.e236d59f4d71d7c774ec9d53285b56f7.jpg: 640x640 1 Crop, 19.3ms
image 7/1000 /kaggle/working/final/Dataset/Unlabelled/agri_0_1028_jpeg.rf.32fc950fe95beacd547893f9915ab5a9.jpg: 640x640 15 Crops, 19.2ms
image 8/1000 /kaggle/working/final/Dataset/Unlabelle

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        130        214      0.931      0.879      0.952      0.755
                  Crop         60        115      0.927      0.887      0.945      0.768
                  Weed         70         99      0.935       0.87      0.959      0.742
Speed: 0.2ms preprocess, 15.1ms inference, 0.1ms loss, 2.5ms postprocess per image
Results saved to /kaggle/working/runs/train/yolo11m_augmented_dataset2
Training finished. Best model saved at: model_pseudo.pt


In [15]:

# --- Part 1: Generate High-Confidence Pseudo-Labels ---

# Load the model trained on the initial pseudo-labels
model = YOLO("model_pseudo.pt") 

# Define paths
UNLABELED_DIR = "/kaggle/working/final/Dataset/Unlabelled"
PSEUDO_LABELS_DIR = "/kaggle/working/yolo_dataset/labels/pseudo_fixmatch" # Use a new directory
os.makedirs(PSEUDO_LABELS_DIR, exist_ok=True)

# Predict on unlabeled images (this is for INFERENCE to get labels, not for training)
results = model.predict(source=UNLABELED_DIR, save=False, verbose=False)

# Save high-confidence predictions as new pseudo-labels
# We can use a stricter confidence threshold here as the model has improved
for result in results:
    img_name = os.path.basename(result.path).replace(".jpg", ".txt")
    label_path = os.path.join(PSEUDO_LABELS_DIR, img_name)
    with open(label_path, "w") as f:
        for box in result.boxes.data:
            cls, x_center, y_center, width, height, conf = box.tolist()
            if conf > 0.90:  # Using a stricter confidence for the second round
                f.write(f"{int(cls)} {x_center} {y_center} {width} {height}\n")

print(f"✅ FixMatch pseudo-labeling complete. Labels saved in: {PSEUDO_LABELS_DIR}")

# --- Part 2: Retrain with the New, Higher-Quality Pseudo-Labels ---

# Add the new pseudo-labels to the main training labels directory
TRAIN_LABELS_DIR = "/kaggle/working/yolo_dataset/labels/train"
for file in os.listdir(PSEUDO_LABELS_DIR):
    if file.endswith(".txt"):
        shutil.copy(os.path.join(PSEUDO_LABELS_DIR, file), TRAIN_LABELS_DIR)

print("✅ New pseudo-labels have been added to the training set.")
train_params = {
    "data": "/kaggle/working/yolo_dataset/data.yaml",  # must include 'train' and 'val'
    "epochs": 100,
    "patience": 10,          # early stopping
    "imgsz": 640,
    "batch": 16,
    "device": "cuda",
    "optimizer": "SGD",
    "lr0": 0.01,
    "lrf": 0.1,
    "weight_decay": 0.0005,
    "momentum": 0.937,
    "workers": 8,
    "project": "runs/train",
    "name": "yolo11m_augmented_dataset",
    "verbose": True
}
# Retrain the model using the combined dataset
model.train(
    **train_params
)

# Save the newly trained FixMatch model
model.save("model_fixmatch.pt")

print("✅ FixMatch model training complete! Weights saved to model_fixmatch.pt")

✅ FixMatch pseudo-labeling complete. Labels saved in: /kaggle/working/yolo_dataset/labels/pseudo_fixmatch
✅ New pseudo-labels have been added to the training set.
Ultralytics 8.3.203 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/yolo_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.1, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=model_pseudo.pt, mom

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        130        214       0.88      0.935      0.956       0.74
                  Crop         60        115       0.88       0.93      0.951      0.752
                  Weed         70         99       0.88      0.939      0.961      0.728
Speed: 0.2ms preprocess, 15.6ms inference, 0.0ms loss, 2.4ms postprocess per image
Results saved to /kaggle/working/runs/train/yolo11m_augmented_dataset3
✅ FixMatch model training complete! Weights saved to model_fixmatch.pt


In [31]:

# ----------------------------
# Load Student Model
# ----------------------------
student_model = YOLO("model_fixmatch.pt").to("cuda")
student_model.train()

# ----------------------------
# Initialize Teacher Model
# ----------------------------
teacher_model = YOLO("model_fixmatch.pt").to("cuda")
teacher_model.model.load_state_dict(student_model.model.state_dict())  # copy weights
teacher_model.eval()

# ----------------------------
# Pseudo-label directory
# ----------------------------
PSEUDO_LABELS_DIR = "/kaggle/working/labels/pseudo/"
os.makedirs(PSEUDO_LABELS_DIR, exist_ok=True)

# ----------------------------
# Optimizer for Student
# ----------------------------
optimizer = torch.optim.AdamW(student_model.model.parameters(), lr=5e-5)

# EMA decay factor
alpha = 0.999

# ----------------------------
# Load Unlabeled Data
# ----------------------------
UNLABELED_DIR = "/kaggle/input/weed-detection-dataset/unlabeled"
image_files = [f for f in os.listdir(UNLABELED_DIR) if f.endswith(".jpg")]

# ----------------------------
# Generate Pseudo-labels
# ----------------------------
for img_file in image_files:
    img_path = os.path.join(UNLABELED_DIR, img_file)

    # Get predictions from Teacher
    teacher_preds = teacher_model.predict(img_path, conf=0.85)

    if len(teacher_preds) == 0:
        continue  # skip if no detections

    teacher_preds = teacher_preds[0]  # first result

    # Save pseudo-labels in YOLO format
    label_file = os.path.join(PSEUDO_LABELS_DIR, img_file.replace(".jpg", ".txt"))
    with open(label_file, "w") as f:
        for i in range(len(teacher_preds.boxes.xywhn)):
            x_center, y_center, width, height = teacher_preds.boxes.xywhn[i].tolist()
            cls = int(teacher_preds.boxes.cls[i].item())
            f.write(f"{cls} {x_center} {y_center} {width} {height}\n")

print("✅ Pseudo-label generation completed.")

# ----------------------------
# Retrain Student Model on pseudo-labeled data
# ----------------------------
student_model.train(
    data="/kaggle/working/yolo_dataset/data.yaml",
    epochs=100,
    imgsz=640,
    batch=4,
    device="cuda",
    save=True,
    patience=10
)

# ----------------------------
# Update Teacher Model with EMA
# ----------------------------
with torch.no_grad():
    for teacher_param, student_param in zip(teacher_model.model.parameters(), student_model.model.parameters()):
        teacher_param.data.mul_(alpha).add_(student_param.data * (1 - alpha))

# ----------------------------
# Save Mean Teacher Student Model
# ----------------------------
student_model.save("student_model_mean_teacher.pt")
teacher_model.save("teacher_model.pt")

print("✅ Mean Teacher Model training completed and saved as 'model_mean_teacher.pt'.")


engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/yolo_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=model_fixmatch.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train11, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, project=None, rect=False, res

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        130        214      0.926       0.91      0.959      0.766
                  Crop         60        115      0.915      0.913      0.958      0.775
                  Weed         70         99      0.937      0.908       0.96      0.757
Speed: 0.2ms preprocess, 15.9ms inference, 0.0ms loss, 3.3ms postprocess per image
Results saved to /kaggle/working/runs/detect/train11

image 1/1 /kaggle/input/weed-detection-dataset/unlabeled/agri_0_5225_jpeg.rf.ab0d818fca3b3b0ef3411dea1a56000f.jpg: 640x640 (no detections), 20.8ms
Speed: 2.4ms preprocess, 20.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /kaggle/input/weed-detection-dataset/unlabeled/agri_0_2390_jpeg.rf.b4af55b63fe3ad51e5ccaa7fae52a15a.jpg: 640x640 1 Weed, 20.9ms
Speed: 2.4ms preprocess, 20.9ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /kaggle/input/weed-detection-dataset/unlabeled/agri_0_2633_jpeg.rf.80320a4fd05d32f7fae265c2d09ef7c4.jpg: 

KeyboardInterrupt: 

In [32]:
# Assuming your training is complete and the best model is saved
# The path to the best model is saved within the run directory
best_model_path = "/kaggle/working/runs/train/yolo11m_augmented_dataset/weights/best.pt"

# Load the best model from the training directory
best_model = YOLO(best_model_path)

# Now save the best model to a new, user-friendly name
best_model.save("model_mean_teacher.pt")

print("✅ Best Mean Teacher Model training completed and saved as 'model_mean_teacher.pt'.")

✅ Best Mean Teacher Model training completed and saved as 'model_mean_teacher.pt'.


In [43]:
import torch
from ultralytics import YOLO
from tabulate import tabulate  # For nice tables

# List your trained models
models_info = [
    {"name": "Model3", "path": "/kaggle/working/model_mean_teacher.pt"},
    {"name": "Model3","path": "/kaggle/working/v11m_50_epochs.pt"}
    # Add more models here if needed
]

# Path to your dataset YAML
data_yaml = "/kaggle/working/yolo_dataset/data.yaml"

# List to store metrics for comparison
comparison_metrics = []

for info in models_info:
    print(f"\nEvaluating {info['name']}...")
    model = YOLO(info["path"]).to("cuda")
    model.eval()
    
    # Run validation
    results = model.val(data=data_yaml)
    res_dict = results.results_dict
    
    # Extract metrics safely
    precision = res_dict.get('metrics/precision(B)', 0)
    recall = res_dict.get('metrics/recall(B)', 0)
    mAP_50_95 = res_dict.get('metrics/mAP50-95(B)', 0)
    
    # Compute F1
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    combined_metric = 0.5 * f1_score + 0.5 * mAP_50_95
    
    comparison_metrics.append({
        "Model": info["name"],
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1_score,
        "mAP50-95": mAP_50_95,
        "Combined": combined_metric
    })

# Print comparative table
print("\n=== Comparative Analysis ===")
print(tabulate(comparison_metrics, headers="keys", tablefmt="fancy_grid", floatfmt=".4f"))



Evaluating Model3...
YOLOv12m summary (fused): 169 layers, 20,106,454 parameters, 0 gradients, 67.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2096.5±748.7 MB/s, size: 158.0 KB)
val: Scanning /kaggle/working/yolo_dataset/labels/val.cache... 130 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 130/130 300.7Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 9/9 2.5it/s 3.7s0.4ss


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        130        214       0.89      0.922      0.954      0.734
                  Crop         60        115       0.85      0.913      0.947      0.756
                  Weed         70         99      0.929      0.931      0.961      0.712
Speed: 2.8ms preprocess, 16.9ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to /kaggle/working/runs/detect/val19

Evaluating Model3...
YOLOv12m summary (fused): 169 layers, 20,106,454 parameters, 0 gradients, 67.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1935.6±370.5 MB/s, size: 138.1 KB)
val: Scanning /kaggle/working/yolo_dataset/labels/val.cache... 130 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 130/130 259.2Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 9/9 2.5it/s 3.7s0.4ss


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        130        214      0.906      0.897      0.955      0.688
                  Crop         60        115      0.874      0.896      0.942      0.706
                  Weed         70         99      0.939      0.899      0.967      0.669
Speed: 2.5ms preprocess, 16.7ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to /kaggle/working/runs/detect/val20

=== Comparative Analysis ===
╒═════════╤═════════════╤══════════╤════════════╤════════════╤════════════╕
│ Model   │   Precision │   Recall │   F1-Score │   mAP50-95 │   Combined │
╞═════════╪═════════════╪══════════╪════════════╪════════════╪════════════╡
│ Model3  │      0.8896 │   0.9220 │     0.9055 │     0.7343 │     0.8199 │
├─────────┼─────────────┼──────────┼────────────┼────────────┼────────────┤
│ Model3  │      0.9063 │   0.8973 │     0.9018 │     0.6876 │     0.7947 │
╘═════════╧═════════════╧══════════╧════════════╧════════════╧════════════╛
